In [1]:
import json
import numpy as np
from bokeh.plotting import figure
from bokeh import embed
from bokeh.io import show, output_notebook
from bokeh.palettes import Category20_20
colors = Category20_20
output_notebook()

Loading BokehJS ...

In [4]:
info = json.load(open('runs/38/info.json'))

In [5]:
c = 0
p = figure(title='CNN', plot_width=800, plot_height=600)

metrics = {}
for k in info['evaluations'][0].keys():
    if k.endswith('_auc'):
        metrics[k] = []

for run in info['evaluations']:
    for m in metrics.keys():
        metrics[m].append(run[m])

for metric, values in metrics.items():
    xs = np.array([i/len(values) for i in range(1, len(values)+1)])
    ys = np.array(values)
    p.line(xs, ys, legend=metric, color=colors[c], line_width=3)
    c += 1

p.xaxis.axis_label = 'train proportion'
p.yaxis.axis_label = 'AUC'
p.legend.location = 'bottom_right'

show(p)